In [1]:
from ngsolve import *
from ngsolve.webgui import Draw
from netgen.occ import *
from netgen.webgui import Draw as Drawgeo
import matplotlib.pyplot as plt
import numpy as np


importing NGSolve-6.2.2301


## Exact Solution:
Solving for E then converting to H via Faraday's law

In [2]:
from scipy.special import jv as besselj
from scipy.special import yv as bessely
from scipy.special import hankel1, hankel2
import numpy as np
from matplotlib import pyplot as plt

omega = 2*np.pi
N_samples = 50
x_extent = np.linspace(-2.5, 2.5, N_samples)
y_extent = x_extent
z_extent = x_extent
xx,yy,zz = np.meshgrid(x_extent, y_extent, z_extent, indexing='ij')

R = np.sqrt(xx**2 + yy**2)
theta = np.arctan2(yy, xx)

# plt.figure()
# plt.imshow(theta, extent=[-2.5,2.5, -2.5, 2.5])
# plt.colorbar()

mask = np.ones(xx.shape)
for yind, yval in enumerate(yy):
    for xind, xval in enumerate(xx):
        if R[xind, yind,0] < 1:
            mask[xind, yind, :] = np.nan

# plt.figure()
# plt.imshow(mask[:,:,10], extent=[-2.5,2.5,-2.5,2.5])


amp = np.zeros(xx.shape)

TE = False
if TE is True:
# TE
    for n in range(30):
        dj=(-besselj(n-1,omega)+besselj(n+1,omega))/2
        dy=(-bessely(n-1,omega)+bessely(n+1,omega))/2
        dh=dj-1j*dy;

        z=omega*R;
        bj=besselj(n,z);
        by=bessely(n,z);
        h=bj-1j*by;
        if n==0:
            amp=amp-(dj/dh)*h*np.cos(n*theta)*(1j**(-n)) * mask
        else:
            amp=amp-2*(dj/dh)*h*np.cos(theta*n)*(1j**(-n)) * mask
            
else:
    for n in range(30):
        if n == 0:
            epsilon_n = 1
        else:
            epsilon_n = 2
        
        # for cylinder radius r=1
        
        jv_over_h2 = besselj(n, omega*1)/hankel1(n, omega*1)
        amp = amp -(1j)**n * epsilon_n * jv_over_h2 * hankel1(n, omega * R) * np.cos(theta*n) * mask
        
        #zz = np.conj(zz) # Don't know why this needs a conjugate. Probably something to do with the direction of the wave Balanis assumed.

E = np.asarray([np.zeros(amp.shape), np.zeros(amp.shape), amp])
E_x = E[0,:,:, :]
E_y = E[1,:,:,:]
E_z = E[2,:,:,:]

dist = 5/N_samples
curlE = np.asarray([np.gradient(E_z,dist, axis=1) , -np.gradient(E_z, dist, axis=0) , np.zeros(zz.shape)])
#curlE = np.asarray([np.gradient(E_z, axis=1) - np.gradient(E_y, axis=2), np.gradient(E_z, axis=0) - np.gradient(E_x, axis=2), np.gradient(E_y, axis=0) - np.gradient(E_x, axis=1)])
H = (-1 / (1j*omega)) * curlE

print(curlE.shape)



mask = np.ones(mask.shape)
for xind in range(mask.shape[0]):
    for yind in range(mask.shape[1]):
        for zind in range(mask.shape[2]):
            if np.isnan(curlE[0, xind,yind,zind]):
                mask[xind, yind, zind] = np.nan
            elif np.isnan(curlE[1, xind,yind, zind]):
                mask[xind, yind, zind] = np.nan
            elif np.isnan(curlE[2, xind,yind, zind]):
                mask[xind, yind, zind] = np.nan

# plt.figure()
# plt.imshow(E_z[:,:,10], extent=[-2.5,2.5, -2.5, 2.5])

Exact = np.asarray([H[0,:,:], H[1,:,:], H[2,:,:]]) * mask


# plt.figure()
# plt.title('H x Field real')
# plt.imshow(H[0,:,:,10].real, cmap='jet', extent=[-2.5,2.5, -2.5, 2.5])
# plt.colorbar()

# plt.figure()
# plt.title('H x Field imag')
# plt.imshow(H[0,:,:,10].imag, cmap='jet', extent=[-2.5,2.5, -2.5, 2.5])
# plt.colorbar()

# plt.figure()
# plt.title('H y Field real')
# plt.imshow(H[1,:,:,10].real, cmap='jet', extent=[-2.5,2.5, -2.5, 2.5])
# plt.colorbar()

# plt.figure()
# plt.title('H y Field imag')
# plt.imshow(H[1,:,:,10].imag, cmap='jet', extent=[-2.5,2.5, -2.5, 2.5])
# plt.colorbar()



# plt.figure()
# plt.title('H Field Quiver Real')
# plt.quiver(xx[0::4, 0::4], yy[0::4,0::4], H[0,0::4,0::4].real, H[1,0::4,0::4].real, np.sqrt(H[0,0::4,0::4].real**2 + H[1,0::4,0::4].imag**2))
# ax = plt.gca()
# #get x and y limits
# x_left, x_right = ax.get_xlim()
# y_low, y_high = ax.get_ylim()
# #set aspect ratio
# ax.set_aspect(abs((x_right-x_left)/(y_low-y_high)))
# plt.colorbar()

# plt.figure()
# plt.title('H Field Quiver imag')
# plt.quiver(xx[0::4, 0::4], yy[0::4,0::4], H[0,0::4,0::4].imag, H[1,0::4,0::4].imag, np.sqrt(H[0,0::4,0::4].imag**2 + H[1,0::4,0::4].imag**2))
# ax = plt.gca()
# #get x and y limits
# x_left, x_right = ax.get_xlim()
# y_low, y_high = ax.get_ylim()
# #set aspect ratio
# ax.set_aspect(abs((x_right-x_left)/(y_low-y_high)))
# plt.colorbar()


(3, 50, 50, 1)


## Generating Mesh

In [3]:
def generate_mesh(h, inner_rect_size, PML_size):
    
    inner_rect=WorkPlane().RectangleC(inner_rect_size,inner_rect_size).Face().Extrude(5).Move((0,0,-2.5))
    scatterer = WorkPlane().Circle(0,0,1).Face().Extrude(5).Move((0,0,-2.5))

    inner_rect.edges.name = 'innerbnd'
    scatterer.edges.name = 'scabnd'

    inner = inner_rect - scatterer

    #Drawgeo(inner)

    wp2=WorkPlane().RectangleC(inner_rect_size+PML_size,inner_rect_size+PML_size).RectangleC(inner_rect_size, inner_rect_size).Reverse()
    outer = wp2.Face().Extrude(5).Move((0,0,-2.5))

    outer.edges.name = 'outerbnd'
    #inner.edges.name = 'innerbnd'
    inner.faces.name ='inner'
    outer.faces.Max(Z).name = 'upperlowerbnd'
    outer.faces.Min(Z).name = 'upperlowerbnd'

    scatterer.faces.name = 'scabnd'

    #Drawgeo(Glue([outer,inner]))

    geo = OCCGeometry(Glue([inner, outer]), dim=3)
    mesh = Mesh(geo.GenerateMesh (maxh=h))
    mesh.Curve(5)
    return mesh

inner_rect_size = 5
PML_size = 5
h = 0.75
mesh = generate_mesh(h, inner_rect_size, PML_size)
Draw(mesh)

print(mesh.GetBoundaries())

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.23…

('inner', 'inner', 'inner', 'inner', 'inner', 'inner', 'scabnd', 'default', 'default', 'default', 'default', 'upperlowerbnd', 'upperlowerbnd')


In [4]:
K = CF((omega, 0))
# phasor = exp(1j * ((K[0] * x) + (K[1] * y)))
# hx = 1 * phasor
# hy = 1 * phasor
# h =  CF((hx, hy, 0))
# #Draw(h, mesh)


phasor = exp(1j * ((K[0] * x) + (K[1] * y)))
ez = 1 * phasor

E = CF((0, 0, ez))

omega = sqrt(K[0]**2 + K[1]**2)
h = CF((E[2].Diff(y) - E[1].Diff(z), (E[2].Diff(x) - E[0].Diff(z)), 0)) * (-1/(1j*omega))
#h = CF((0, E[2].Diff(x), 0))
#Draw(h[0], mesh)
#Draw(h[1], mesh)

In [5]:
# Neumann BC:
curlh = CF((0, 0, (h[1].Diff(x) - h[0].Diff(y))))
# normal = specialcf.normal(dim=3)
# n_cross_curlh = Cross(normal, -curlh)
# #n_cross_curlh_2d = CF((n_cross_curlh[0], n_cross_curlh[1]))
# n_cross_curlh2 = Cross(normal, n_cross_curlh)
# n_cross_curlh2_1d = CF((n_cross_curlh2[2], 0))
#Draw(normal, mesh)

#func_domain2 = CoefficientFunction ([-specialcf.normal(3) if mat== "inner" else None for mat in mesh.GetMaterials()])
#mesh.GetBoundaries()
#Draw(func_domain2, mesh)
#Draw(curlh[2], mesh)
ang = atan2(y,x)
#Draw(ang, mesh)
normal = CF((cos(ang), sin(ang), 0))
#Draw(normal[0], mesh)
#Draw(normal[1], mesh)
n_cross_curlh = Cross(normal, curlh)
n_cross_curlh_2d = CF((n_cross_curlh[0], n_cross_curlh[1]))
#Draw(n_cross_curlh2[0], mesh)
#Draw(n_cross_curlh2[1], mesh)
#Draw(n_cross_curlh2[2], mesh)


## With PML

In [6]:
def absval(x):
    return sqrt(x**2)

d = inner_rect_size
z_x = IfPos(absval(x) - d/2, x + (1j * (( absval(x) - d)/0.5)**1) * x, x) # returns z_j if |x|>2.5 else returns x
z_y = IfPos(absval(y) -d/2, y + (1j * (( absval(y) - d)/0.5)**1) * y, y) # returns z_j if |y|>2.5 else returns y
z_z = z
#Draw(z_x.imag, mesh)
#Draw(z_y.imag, mesh)

In [7]:
dzx = z_x.Diff(x)
dzy = z_y.Diff(y)
dzz = z_z.Diff(z)

#Draw(dzx.real, mesh)
#Draw(dzy.real, mesh)

#Draw(dzx.imag, mesh)
#Draw(dzy.imag, mesh)

dz_tot = dzx * dzy
Draw(dz_tot.imag,mesh)



WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.23…

BaseWebGuiScene

In [8]:
def With_PML(mesh, p):

    fes = HCurl(mesh, order=p, complex=True)
    u = fes.TrialFunction()
    v = fes.TestFunction()
    
    omega = sqrt(K[0]**2 + K[1]**2)
    
    f = dzy / dzx
    Lambda = CF((f,0,0, 0,1/f,0, 0,0,dzy*dzx), dims=(3,3))
    Lambda_inv = CF((1/f,0,0, 0,f,0, 0,0,(1/(dzy*dzx))), dims=(3,3))
    #Lambda =1
    scat = GridFunction(fes)
    #scat.Set(CF((-h[0],-h[1])), definedon='scabnd')
    #scat.Set(CF((0,0)), BND)
     
    
    a = BilinearForm(fes, symmetric=True)
    a += ((Lambda_inv) *curl(u))*curl(v)*dx - omega**2*(Lambda *u) *v *dx
    a.Assemble()
    
    b = LinearForm(fes)
    b += n_cross_curlh * v.Trace() * ds('scabnd')
    #b += -n_cross_curlh * v.Trace() * ds('upperlowerbnd')
    b.Assemble()
    
    
    r = b.vec.CreateVector()
    r = b.vec - a.mat * scat.vec
    scat.vec.data += a.mat.Inverse(freedofs=fes.FreeDofs()) * r
    return scat, fes.ndof


scat, _ = With_PML(mesh, 3)

Draw(scat[0].real, mesh)
# Draw(scat[0].imag, mesh)

Draw(scat[1].real, mesh)
# Draw(scat[1].imag, mesh)


WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.23…

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.23…

BaseWebGuiScene

## Comparing with Exact Solution

In [9]:

from ngsolve import *


def compute_on_meshgrid(xx, yy, zz, scat, mask):
    scat_numpy_x = np.zeros(xx.ravel().shape, dtype=complex)
    scat_numpy_y = np.zeros(xx.ravel().shape, dtype=complex)
    scat_numpy_z = np.zeros(xx.ravel().shape, dtype=complex)

    for indx, xy in enumerate(zip(xx.ravel(), yy.ravel(), zz.ravel())):
        R = np.sqrt(xy[0]**2 + xy[1]**2)
        if R >= 1:
            scat_numpy_x[indx] = scat(mesh(xy[0], xy[1], xy[2]))[0]
            scat_numpy_y[indx] = scat(mesh(xy[0], xy[1], xy[2]))[1]

    scat_numpy_x = scat_numpy_x.reshape(xx.shape) * mask
    scat_numpy_y = scat_numpy_y.reshape(xx.shape) * mask
    return scat_numpy_x, scat_numpy_y

scat_numpy_x, scat_numpy_y = compute_on_meshgrid(xx, yy, zz, scat, mask)

plt.figure()
plt.imshow(scat_numpy_x[:,:,10].real, cmap='jet')
plt.colorbar()
plt.title('FEM Real X')
"""
plt.figure()
plt.imshow(scat_numpy_x[:,:,10].imag, cmap='jet')
plt.colorbar()
plt.title('FEM Imag X')

plt.figure()
plt.imshow(scat_numpy_y[:,:,10].real, cmap='jet')
plt.colorbar()
plt.title('FEM Real Y')

plt.figure()
plt.imshow(scat_numpy_y[:,:,10].imag, cmap='jet')
plt.colorbar()
plt.title('FEM Imag Y')
"""

plt.figure()
plt.imshow(scat_numpy_x[:,:,0].real, cmap='jet')
plt.colorbar()
plt.title('FEM Real X 1')

plt.figure()
plt.imshow(scat_numpy_x[:,:,9].real, cmap='jet')
plt.colorbar()
plt.title('FEM Real X 10')

plt.figure()
plt.imshow(scat_numpy_x[:,:,14].real, cmap='jet')
plt.colorbar()
plt.title('FEM Real X 15')

plt.figure()
plt.imshow(scat_numpy_x[:,:,49].real, cmap='jet')
plt.colorbar()
plt.title('FEM Real X 50')

IndexError: index 10 is out of bounds for axis 2 with size 1

<Figure size 640x480 with 0 Axes>

In [ ]:
def compute_err(exact, approx):
    return np.linalg.norm((exact - approx)) / np.linalg.norm(exact)

Exact = np.asarray([Exact[0,:,:,:], Exact[1,:,:,:]])
for h in [0.5, 0.25, 0.125, 0.065]:
    error = []
    ndof = []
    mesh = generate_mesh(h, inner_rect_size, PML_size)
    for p in [0,1,2,3,4,5,6]:
        print(f'Solving for p={p}')
        scat_pml, nd_pml =  With_PML(mesh, p)
        scat_pml_numpy_x, scat_pml_numpy_y = compute_on_meshgrid(xx, yy, zz, scat_pml, mask)
        scat_pml_numpy = np.asarray([scat_pml_numpy_x, scat_pml_numpy_y])
        scat_pml_numpy_no_nans = scat_pml_numpy[~np.isnan(scat_pml_numpy)]
        exact_no_nans = Exact[~np.isnan(Exact)]
    
        print(Exact.shape)
        print(Exact[~np.isnan(Exact)].shape)
        print(scat_pml_numpy.shape)
        print(scat_pml_numpy[~np.isnan(scat_pml_numpy)].shape)
        
        err = compute_err((np.asarray(exact_no_nans)), np.asarray(scat_pml_numpy_no_nans))
        ndof += [nd_pml]
        error += [err]
    
    plt.figure(1)
    plt.loglog(ndof, error, label=f'h={h}')
    plt.ylabel('Relative Error')
    plt.xlabel('NDOF')
    plt.legend()
    print('')


Note that since we are computing the exact solution for $\boldsymbol{H}$ via taking the curl of the exact $\boldsymbol{E}$ field where gradients are approximated using finite differences, the exact $\boldsymbol{H}$ field is really an approximate solution dependent upon the phase and frequency of the electric field and the number of discrete samples.

Increasing <i>N_samples</i> improves the accuracy of $\boldsymbol{H}$ and by extension the error reported above.